In [1]:
from jl_song_data import SongData 
import pandas as pd 
from jl_chord_cleaning import ChordDatasetCleaner

In [2]:
data = SongData('../data/chords_clean_1_2_3_4_5_6.csv')

In [3]:
df = data.df

In [4]:
df

,url,name,decade,genre,chords,uuid
0,https://tabs.ultimate-guitar.com/tab/10000_man...,Dont Talk,1980s,Folk,"['D', 'Dmaj7', 'D', 'Dmaj7', 'D', 'Dmaj7', 'D'...",c639eb23-fefd-4263-af20-3f78f110edcd
1,https://tabs.ultimate-guitar.com/tab/10000_man...,Whats The Matter Here,1980s,Folk%%Folk,"['G', 'G', 'C', 'D', 'G', 'G', 'G', 'G', 'G', ...",828ba8e4-d791-4403-a4cc-5ca4a1c0cc1a
2,https://tabs.ultimate-guitar.com/tab/1002296,Limón Y Sal (ver 2),2000s,Pop%%Folk,"['F', 'G', 'Am', 'G', 'C', 'F', 'F', 'G', 'Am'...",bd5b9396-8d6a-49ee-a738-6577cf310783
3,https://tabs.ultimate-guitar.com/tab/1054759,Snälla Bli Min,2010s,Electronic,"['Asus4', 'G', 'Bm7/A', 'Bm', 'D/F#', 'A', 'As...",ef979fca-e68a-4d8f-bd27-3d048516dc76
4,https://tabs.ultimate-guitar.com/tab/1055161,Time To Say Goodbye Con Te Partirò,1990s,Pop%%Classical%%Pop,"['G', 'D', 'Em', 'C', 'G', 'D', 'Em', 'C', 'G'...",d42333bf-e925-4ad1-a4b9-bfe2e2df209e
...,...,...,...,...,...,...
14109,https://tabs.ultimate-guitar.com/tab/ziggy_mar...,Personal Revolution,2010s,Reggae,"['Bm', 'A', 'Bm', 'A', 'Bm', 'A', 'Em', 'G', '...",089f0438-4e4d-44dd-83e5-5dd929753f0f
14110,https://tabs.ultimate-guitar.com/tab/ziggy_mar...,Shalom Salaam,2000s,Reggae,"['Gm', 'Bb', 'F', 'Gm', 'Gm', 'Bb', 'F', 'Gm',...",149bb9e5-fb06-4d73-a69b-b5d9573d2993
14111,https://tabs.ultimate-guitar.com/tab/ziggy_mar...,True To Myself,2000s,Reggae%%Reggae%%Reggae,"['A', 'E', 'Bm', 'D', 'A', 'E', 'Bm', 'D', 'A'...",c0d75c52-ae53-4b47-bc8e-a723ba542d00
14112,https://tabs.ultimate-guitar.com/tab/ziggy_mar...,True To Myself (ver 2),2000s,Reggae,"['A', 'E', 'Bm', 'A', 'E', 'Bm', 'A', 'E', 'Bm...",29047e58-5391-4875-ab94-3a5a7fdbcd0e


In [5]:
df['chords'] = df['chords'].apply(eval)

In [6]:
def get_all_chords(dataframe):
    all_chords = []

    for chord_list in dataframe['chords']:
        all_chords.extend(chord_list)
    
    return all_chords

In [7]:
all_chords = get_all_chords(df)
len(all_chords)

1208088

In [8]:
def get_unique_chords(dataframe):
    unique_chords = set([])

    for chord_list in dataframe['chords']:
        unique_chords = unique_chords.union(set(chord_list))

    unique_chords = list(unique_chords)  
    return unique_chords

In [9]:
unique_chords = get_unique_chords(df)
len(unique_chords)

2235

In [10]:
from pychord_jl import Chord
def replace_german_system(input):
    if input[0] == 'H':
        return 'Bb'+input[1:]
    
    return input

def replace_6_9(input):
    return (input
            .replace('6/9', '6_9')
            .replace('7/9', '7_9')
            .replace('7/5', '7_5')
            .replace('7/6', '7_6')
           )

def replace_enharmonic(input):
    return (input
            .replace('E#', 'F')
            .replace('B#', 'C')
            .replace('Fb', 'E')
            .replace('Cb', 'B')
           )

def replace_errors(input):
    return (input
            .replace('G(2)', 'G')
            .replace('A*', 'A')
            .replace('(III)', '')
            .replace('Gm#', 'G#m')
            .replace('Fm#', 'F#m')
            .replace('Am#', 'A#m')
           )

def parse(input):
    input = replace_german_system(input)
    input = replace_6_9(input)
    input = replace_enharmonic(input)
    input = replace_errors(input)
    
    try:
        return Chord(input)
    except:
        return None

    
    

In [11]:
black_list = ['Go', 'AM']

In [12]:
parsed_chords = [ (parse(chord_str),chord_str) for chord_str in unique_chords if chord_str not in black_list]

In [13]:
def count_appearances(chord_str):
    return len([chord for chord in all_chords if chord == chord_str])

In [14]:
not_found = [ (count_appearances(x[1]),x[1]) for x in parsed_chords if x[0] is None]
len(not_found)

255

In [15]:
import functools 
total_not_found = functools.reduce(lambda x,y:x+y, [ count_appearances(x[1]) for x in parsed_chords if x[0] is None])

In [16]:
not_found

[(5, 'LA#'),
 (3, 'F#6 (add9)'),
 (3, 'Fmaj7+11'),
 (3, 'F#madd6'),
 (6, 'B7/13'),
 (1, 'F(6)'),
 (6, 'Bbmaj7sus4/Eb'),
 (2, 'F(maj7)'),
 (1, 'D(4)'),
 (1, 'Dm7+5'),
 (4, 'Gm7b5b9'),
 (6, 'B11b9'),
 (1, 'C8'),
 (1, 'Dmadd4'),
 (2, 'E7(b13)'),
 (1, 'Bsus4dim7'),
 (4, 'Em7(VII)'),
 (1, 'C6add4'),
 (6, 'FMAJ7'),
 (2, 'C(maj7)'),
 (4, 'Bb6a'),
 (1, 'B11b9/E'),
 (12, 'Emmaj9'),
 (1, 'Amadd6'),
 (4, 'FmMaj7'),
 (5, 'D#(b5)'),
 (7, 'E7/9b'),
 (1, 'F#(2)/A#'),
 (1, 'Eb(dim7)'),
 (2, 'Cadd#11'),
 (5, 'Cmaj7#5'),
 (9, 'F#m7#5'),
 (1, 'Gmaj7+9'),
 (7, 'G(7+)'),
 (2, 'F(2)/A'),
 (2, 'Bbmaj9add13'),
 (2, 'E5#11'),
 (1, 'Bm(add11)'),
 (4, 'Gmaj7sus4/C'),
 (15, 'BB'),
 (2, 'E(3)'),
 (3, 'Cm*'),
 (4, 'Eaddb13'),
 (1, 'Gm7+5'),
 (5, 'C#9(#5)'),
 (7, 'A(5)'),
 (1, 'Bb7(b5)'),
 (1, 'C#(IV)'),
 (1, 'Ab(Add9)'),
 (3, 'Cm7/9'),
 (16, 'G/H'),
 (1, 'Fdim3'),
 (1, 'G# (IV)'),
 (1, 'B5b'),
 (4, 'D9/+7'),
 (2, 'A(major)'),
 (17, 'G#(b5)'),
 (2, 'B7*'),
 (3, 'F#7b9sus4'),
 (1, 'Am-5'),
 (4, 'G7s4'),
 (2, 'Cmaj7+5

In [17]:
not_recognised_chords = [chord[1] for chord in not_found]
not_recognised_chords

['LA#',
 'F#6 (add9)',
 'Fmaj7+11',
 'F#madd6',
 'B7/13',
 'F(6)',
 'Bbmaj7sus4/Eb',
 'F(maj7)',
 'D(4)',
 'Dm7+5',
 'Gm7b5b9',
 'B11b9',
 'C8',
 'Dmadd4',
 'E7(b13)',
 'Bsus4dim7',
 'Em7(VII)',
 'C6add4',
 'FMAJ7',
 'C(maj7)',
 'Bb6a',
 'B11b9/E',
 'Emmaj9',
 'Amadd6',
 'FmMaj7',
 'D#(b5)',
 'E7/9b',
 'F#(2)/A#',
 'Eb(dim7)',
 'Cadd#11',
 'Cmaj7#5',
 'F#m7#5',
 'Gmaj7+9',
 'G(7+)',
 'F(2)/A',
 'Bbmaj9add13',
 'E5#11',
 'Bm(add11)',
 'Gmaj7sus4/C',
 'BB',
 'E(3)',
 'Cm*',
 'Eaddb13',
 'Gm7+5',
 'C#9(#5)',
 'A(5)',
 'Bb7(b5)',
 'C#(IV)',
 'Ab(Add9)',
 'Cm7/9',
 'G/H',
 'Fdim3',
 'G# (IV)',
 'B5b',
 'D9/+7',
 'A(major)',
 'G#(b5)',
 'B7*',
 'F#7b9sus4',
 'Am-5',
 'G7s4',
 'Cmaj7+5/G#',
 'A6sus4',
 'G-',
 'DmMaj7/C#',
 'F#m7+5',
 'Db6m',
 'Em7/9',
 'EbmaucaDOr',
 'F#7add4',
 'D(II)',
 'Ebmaj7(#5)',
 'C#m7(b5)',
 'Em+',
 'RE',
 'Eb(maj7)',
 'Am7+5',
 'Dm6/9',
 'Bbmaj7-5',
 'Am/Fnat',
 'AbanDOnamrme',
 'G7+5-9',
 'Am7+',
 'E*',
 'C(2)',
 'AmMaj7',
 'G#(IV)',
 'Bbadd#11',
 'Bm7/9',
 'Fmaj7/F

In [18]:
def contains_not_recognised_chord(chords):
    return any([not_recognised in chords for not_recognised in not_recognised_chords])

In [19]:
print(f'{100 * len(df[df["chords"].apply(contains_not_recognised_chord)]) / len(df)}% of songs with no recognised chords')

1.5303953521326343% of songs with no recognised chords


In [20]:
print(f'{100 * total_not_found / len(all_chords)}% of chords not recognised')

0.08592089317996703% of chords not recognised


In [21]:
not_found.sort(key = lambda x: x[0], reverse = True) 

In [22]:
not_found

[(37, 'G(b5)'),
 (27, 'AmMaj7'),
 (27, 'Emb6'),
 (22, 'RE'),
 (21, 'Em6add9'),
 (21, 'D*'),
 (19, 'F#7add4'),
 (19, 'C*'),
 (17, 'G#(b5)'),
 (17, 'Fadd#11'),
 (16, 'G/H'),
 (16, 'CaddG'),
 (15, 'BB'),
 (15, 'E(2)'),
 (14, 'C(2)'),
 (14, 'Fmaj7-5/A'),
 (14, 'D(2)'),
 (12, 'Emmaj9'),
 (12, 'Cadd#9'),
 (12, 'FMaj9'),
 (11, 'Fmaj9#11'),
 (11, 'A(V)'),
 (11, 'F(b5)'),
 (10, 'D7#'),
 (9, 'F#m7#5'),
 (9, 'D-'),
 (9, 'Aadd#11'),
 (9, 'Cma9'),
 (8, 'C#b7b9'),
 (8, 'Bmadd4'),
 (7, 'E7/9b'),
 (7, 'G(7+)'),
 (7, 'A(5)'),
 (7, 'Bbmaj7-5'),
 (7, 'Dm(add9)/A'),
 (7, 'F+7'),
 (7, 'C+9'),
 (7, '(E)'),
 (7, 'B(2)'),
 (7, 'DO'),
 (6, 'B7/13'),
 (6, 'Bbmaj7sus4/Eb'),
 (6, 'B11b9'),
 (6, 'FMAJ7'),
 (6, 'Am/Fnat'),
 (6, 'G7+5-9'),
 (6, 'B7/13b'),
 (6, 'A7(b13)'),
 (6, 'A7+5+9'),
 (6, 'G(maj7)'),
 (6, 'C(7b9)'),
 (6, 'Dmaddb9'),
 (5, 'LA#'),
 (5, 'D#(b5)'),
 (5, 'Cmaj7#5'),
 (5, 'C#9(#5)'),
 (5, 'G-'),
 (5, 'Em+'),
 (5, 'E*'),
 (5, 'C7#5#9'),
 (5, 'Bb-'),
 (5, 'F(V)'),
 (5, 'Bb7#5#9'),
 (5, 'Dm(V)'),
 (5, 'A

In [23]:
def any_not_found(chords):
    return len([chord for chord in not_found if chord[1] in chords]) > 0

In [24]:
any_not_found(['A'])

False

In [25]:
any_not_found(['Go'])

False

In [26]:
len(df[df['chords'].apply(lambda x: any_not_found(x))]['url'])

216

In [27]:
(df[df['chords'].apply(lambda x: 'D*' in x)]['url']).iloc[0]

'https://tabs.ultimate-guitar.com/tab/gustavo-cerati/hoy-ya-no-soy-yo-chords-1580025'

In [28]:
Chord('A7/G').components()

['G', 'A', 'C#', 'E', 'G']

In [29]:
from pychord_jl.constants import QUALITY_DICT

In [30]:
[list(item[1]) for item in QUALITY_DICT.items()]

[[0, 4],
 [0, 7],
 [0, 7],
 [0, 7],
 [0, 4, 7],
 [0, 4, 7],
 [0, 4, 7],
 [0, 3, 7],
 [0, 3, 7],
 [0, 3, 7],
 [0, 3, 6],
 [0, 3, 6],
 [0, 3, 6],
 [0, 4, 8],
 [0, 4, 8],
 [0, 4, 8],
 [0, 4, 8],
 [0, 2, 7],
 [0, 2, 7],
 [0, 5, 7],
 [0, 5, 7],
 [0, 5, 7],
 [0, 4, 7, 9],
 [0, 4, 7, 9],
 [0, 4, 7, 10],
 [0, 4, 7, 10],
 [0, 4, 7, 10],
 [0, 4, 6, 10],
 [0, 4, 6, 10],
 [0, 4, 6, 10],
 [0, 4, 6, 11],
 [0, 4, 8, 10],
 [0, 4, 8, 10],
 [0, 4, 8, 10],
 [0, 4, 8, 10],
 [0, 2, 7, 9],
 [0, 2, 7, 10],
 [0, 2, 7, 10],
 [0, 5, 7, 10],
 [0, 5, 7, 10],
 [0, 5, 7, 10],
 [0, 5, 7, 10],
 [0, 3, 7, 9],
 [0, 3, 7, 10],
 [0, 3, 7, 10],
 [0, 3, 7, 10],
 [0, 3, 7, 11],
 [0, 3, 7, 10],
 [0, 3, 7, 10],
 [0, 3, 7, 10],
 [0, 3, 6, 10],
 [0, 3, 6, 10],
 [0, 3, 6, 9],
 [0, 4, 7, 11],
 [0, 4, 7, 11],
 [0, 4, 7, 11],
 [0, 4, 7, 11],
 [0, 4, 7, 11],
 [0, 4, 8, 11],
 [0, 3, 7, 11],
 [0, 3, 7, 11],
 [0, 3, 7, 11],
 [0, 4, 7, 14],
 [0, 4, 7, 14],
 [0, 3, 7, 14],
 [0, 4, 7, 14],
 [0, 2, 4, 7],
 [0, 4, 5, 7],
 [0, 4, 7, 17],
 [0

In [31]:
functools.reduce(lambda list1,list2: list(set([*list1, *list2])), [list(item[1]) for item in QUALITY_DICT.items()])

[0, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13, 14, 15, 17, 18, 20, 21]

In [32]:
QUALITY_DICT.

SyntaxError: invalid syntax (<ipython-input-32-d5822689f152>, line 1)